In [1]:
import pathlib
from datetime import datetime
from typing import List, Tuple, Union, Dict

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Column

from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window


In [2]:
# master configuration to use only 4 CPU cores
spark = SparkSession.builder.master("local[4]").getOrCreate()

# basic configuration to use only a reasonable number of partitions
spark.conf.set("spark.sql.shuffle.partition", 4)

# configuration to work in UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/31 16:37:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/31 16:37:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
path = "/Users/emilianofrigo/Downloads/1693460174008_0.jsonl"

# Read JSONL file
df = (
    spark
    .read
    # .option("multiline", "true")
    .json(path)
)


In [11]:
df.printSchema()

root
 |-- _airbyte_ab_id: string (nullable = true)
 |-- _airbyte_data: struct (nullable = true)
 |    |-- _metadata: struct (nullable = true)
 |    |    |-- createdTime: string (nullable = true)
 |    |    |-- dataEndTime: string (nullable = true)
 |    |    |-- dataStartTime: string (nullable = true)
 |    |    |-- marketplaceIds: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- processingEndTime: string (nullable = true)
 |    |    |-- processingStartTime: string (nullable = true)
 |    |    |-- processingStatus: string (nullable = true)
 |    |    |-- reportDocumentId: string (nullable = true)
 |    |    |-- reportId: string (nullable = true)
 |    |    |-- reportType: string (nullable = true)
 |    |-- adjustment-id: string (nullable = true)
 |    |-- amount: string (nullable = true)
 |    |-- amount-description: string (nullable = true)
 |    |-- amount-type: string (nullable = true)
 |    |-- currency: string (nullable = true)
 |   

In [12]:
df.show(truncate=False)

+------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|_airbyte_ab_id                      |_airbyte_data                                                                                                                                                                                                         

In [14]:
df.select("_airbyte_data._metadata.reportId").distinct().show(10)

+------------+
|    reportId|
+------------+
|838773019599|
|838768019599|
+------------+



In [7]:
df.select("_airbyte_data.posted-date").distinct().show()

+-----------+
|posted-date|
+-----------+
| 09.08.2023|
| 03.08.2023|
| 02.08.2023|
| 12.08.2023|
| 13.08.2023|
| 06.08.2023|
| 04.08.2023|
| 08.08.2023|
| 10.08.2023|
| 16.08.2023|
| 14.08.2023|
| 07.08.2023|
| 11.08.2023|
| 15.08.2023|
| 05.08.2023|
|           |
| 2023-08-06|
| 2023-08-04|
| 2023-08-07|
| 2023-08-05|
+-----------+
only showing top 20 rows



23/08/24 22:47:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1015459 ms exceeds timeout 120000 ms
23/08/24 22:47:14 WARN SparkContext: Killing executors is not supported by current scheduler.


In [17]:
df.select("_airbyte_data._metadata.report_id").show(truncate=False)


+------------------------------------+
|report_id                           |
+------------------------------------+
|9acbc73c-9b7c-4a3c-a00a-003c6af19a47|
|9acbc73c-9b7c-4a3c-a00a-003c6af19a47|
+------------------------------------+



23/07/24 20:08:48 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1088666 ms exceeds timeout 120000 ms
23/07/24 20:08:48 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/24 20:08:49 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at s